<a href="https://colab.research.google.com/github/InterDiz/asu/blob/main/keras_inference_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/keras_inference"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/keras_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Начните работу с Gemma с помощью KerasNLP

В этом руководстве показано, как начать работу с Gemma с помощью [KerasNLP](https://keras.io/keras_nlp/). Gemma — это семейство легких современных открытых моделей, созданных на основе тех же исследований и технологий, которые использовались для создания моделей Gemini. KerasNLP — это набор моделей обработки естественного языка (NLP), реализованных в [Keras](https://keras.io/) и работающих на JAX, PyTorch и TensorFlow.

В этом руководстве вы будете использовать Gemma для генерации текстовых ответов на несколько запросов. Если вы новичок в Keras, вам может быть интересно прочитать [Начало работы с Keras](https://keras.io/getting_started/) перед началом работы, но это не обязательно. Вы узнаете больше о Keras по мере работы с этим руководством.

## Setup

### Выберите бэкенд

Keras — это высокоуровневый многофреймворковый API глубокого обучения, разработанный для простоты и удобства использования. [Keras 3](https://keras.io/keras_3) позволяет выбрать бэкенд: TensorFlow, JAX или PyTorch. Все три подойдут для этого руководства.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

### Import требуемых библиотек Python

In [ ]:
import keras
import keras_nlp

## Создать модель

KerasNLP предоставляет реализации многих популярных [архитектур моделей](https://keras.io/api/keras_nlp/models/). В этом руководстве вы создадите модель с использованием `GemmaCausalLM`, сквозной модели Gemma для моделирования каузального языка. Модель каузального языка предсказывает следующий токен на основе предыдущих токенов.

Создайте модель с использованием метода `from_preset`:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("hf://google/gemma-1.1-2b-it-keras", )

model.weights.h5:  22%|##1       | 1.10G/5.01G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocabulary.spm:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Функция `GemmaCausalLM.from_preset()` создает модель из предустановленной архитектуры и весов. В приведенном выше коде строка `"gemma2_2b_en"` определяет предустановленную модель Gemma 2 2B с 2 миллиардами параметров. Также доступны модели Gemma с [7B, 9B и 27B параметрами](/gemma/docs/get_started#models-list). Вы можете найти строки кода для моделей Gemma в их списках **Model Variation** на [Kaggle](https://www.kaggle.com/models/google/gemma).


Примечание: Для запуска более крупных моделей в Colab вам нужен доступ к премиум-GPU, доступным в платных планах. В качестве альтернативы вы можете выполнять выводы с помощью блокнотов Kaggle или проектов Google Cloud.


Используйте `summary` чтобы получить информацию о модели:

In [ ]:
gemma_lm.backbone.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

Как видно из сводки, модель имеет 2,6 миллиарда обучаемых параметров.

Примечание: При наименовании модели («2B») слой эмбеддингов не учитывается в числе параметров.

## Сгенерировать текст

Теперь пришло время сгенерировать текст! Модель имеет метод `generate`, который генерирует текст на основе подсказки. Необязательный аргумент `max_length` указывает максимальную длину сгенерированной последовательности.

Попробуйте с промптом `"что такое keras в 3-х пунктах списка?"`.

In [ ]:
gemma_lm.generate("что такое keras в 3-х пунктах списка?", max_length=255)

KeyboardInterrupt: 

Попробуйте вызвать `generate` еще раз с другим промптом.

In [ ]:
gemma_lm.generate("Вселенная - это", max_length=255)

Если вы работаете на JAX или TensorFlow backends, вы заметите, что второй вызов `generate` возвращает почти мгновенно. Это происходит потому, что каждый вызов `generate` для заданного размера пакета и `max_length` компилируется с помощью XLA. Первый запуск затратен, но последующие запуски намного быстрее.

Вы также можете предоставлять пакетные запросы, используя список в качестве входных данных:

In [ ]:
gemma_lm.generate(
    ["what is keras in 3 bullet points?",
     "The universe is"],
    max_length=64)

### Необязательно: попробуйте другой сэмплер

Вы можете управлять стратегией генерации для `GemmaCausalLM`, установив аргумент `sampler` в `compile()`. По умолчанию будет использоваться сэмплер [`"greedy"`](https://keras.io/api/keras_nlp/samplers/greedy_sampler/#greedysampler-class).

В качестве эксперимента попробуйте установить стратегию [`"top_k"`](https://keras.io/api/keras_nlp/samplers/top_k_sampler/):

In [ ]:
gemma_lm.compile(sampler="top_k")
gemma_lm.generate("Вселенная - это", max_length=64)

В то время как жадный алгоритм по умолчанию всегда выбирает токен с наибольшей вероятностью, алгоритм top-K случайным образом выбирает следующий токен из токенов с наибольшей вероятностью K.

Вам не нужно указывать сэмплер, и вы можете проигнорировать последний фрагмент кода, если он не полезен для вашего варианта использования. Если вы хотите узнать больше о доступных сэмплерах, см. [Сэмплеры](https://keras.io/api/keras_nlp/samplers/).

In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

## Тонкая настройка LoRA
Чтобы получить лучшие ответы от модели, настройте модель с помощью адаптера низкого ранга (LoRA) с использованием набора данных Databricks Dolly 15k.

Ранг LoRA определяет размерность обучаемых матриц, которые добавляются к исходным весам LLM. Он контролирует выразительность и точность тонкой настройки.

Более высокий ранг означает, что возможны более подробные изменения, но также означает больше обучаемых параметров. Более низкий ранг означает меньшие вычислительные затраты, но потенциально менее точную адаптацию.

В этом руководстве используется ранг LoRA 4. На практике начните с относительно небольшого ранга (например, 4, 8, 16). Это вычислительно эффективно для экспериментов. Обучите свою модель с этим рангом и оцените улучшение производительности для вашей задачи. Постепенно увеличивайте ранг в последующих испытаниях и посмотрите, повысит ли это производительность.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

In [ ]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
keras.mixed_precision.set_global_policy('mixed_bfloat16')

In [ ]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Теперь модель рекомендует места для посещения в Европе.

ELI5 Photosynthesis Prompt

In [ ]:
prompt = template.format(
    instruction="Объясни процесс фотосинтеза так, чтобы ребенок мог его понять.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Теперь модель объясняет фотосинтез более простыми словами.

Обратите внимание, что в целях демонстрации этот урок настраивает модель на небольшом подмножестве набора данных всего для одной эпохи и с низким значением ранга LoRA. Чтобы получить лучшие ответы от настроенной модели, вы можете поэкспериментировать с:

Увеличением размера набора данных для настройки
Обучением для большего количества шагов (эпох)
Установкой более высокого ранга LoRA
Изменением значений гиперпараметров, таких как learning_rate и weight_decay.

In [ ]:
gemma_lm.summary()

## What's next

In this tutorial, you learned how to generate text using KerasNLP and Gemma. Here are a few suggestions for what to learn next:

* Learn how to [finetune a Gemma model](https://ai.google.dev/gemma/docs/lora_tuning).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn about [Gemma integration with Vertex AI](https://ai.google.dev/gemma/docs/integrations/vertex)
* Learn how to [use Gemma models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).